In [1]:
# this script is intended to help generate install_dependecy scripts for different platforms


In [3]:
import os
import glob
import json
import datetime

In [8]:

current_date

'2022-08-11 16:47:16.253866'

In [32]:
def bash_header(install_prefix='/opt'):
    current_year = datetime.datetime.now().year
    current_date = str(datetime.datetime.now())
    return '''#!/usr/bin/env bash
# (c) Tuplex team 2017-{}
# auto-generated on {}
# install all dependencies required to compile tuplex + whatever is needed for profiling
# everything will be installed to {} by default'''.format(current_year, current_date, install_prefix) + \
'\n' + '''
# need to run this with root privileges
if [[ $(id -u) -ne 0 ]]; then
  echo "Please run this script with root privileges"
  exit 1
fi

export DEBIAN_FRONTEND=noninteractive

PREFIX={{PREFIX:-{}}}

echo ">> Installing packages into ${{PREFIX}}"
mkdir -p $PREFIX && chmod 0755 $PREFIX
'''.format(install_prefix, )

In [33]:
print(bash_header())

#!/usr/bin/env bash
# (c) Tuplex team 2017-2022
# auto-generated on 2022-08-11 16:52:13.087160
# install all dependencies required to compile tuplex + whatever is needed for profiling
# everything will be installed to /opt by default

# need to run this with root privileges
if [[ $(id -u) -ne 0 ]]; then
  echo "Please run this script with root privileges"
  exit 1
fi

export DEBIAN_FRONTEND=noninteractive

PREFIX={PREFIX:-/opt}

echo ">> Installing packages into ${PREFIX}"
mkdir -p $PREFIX && chmod 0755 $PREFIX



In [34]:
# configure here which versions to use for each platform

CMAKE_VERSION='3.23.2'
YAMLCPP_VERSION='0.6.3'
CELERO_VERSION='2.6.0'
ANTLR_VERSION='4.8'
AWSSDK_VERSION='1.9.133'
AWSLAMBDACPP_VERSION='0.2.6'
PCRE2_VERSION='10.39'
PROTOBUF_VERSION='3.12'
CLOUDPICKLE_VERSION='<2.0.0'
BOOST_VERSION='1.70.0'

In [37]:
def apt_dependencies():
    return 'apt update -y\n' + \
'''
apt-get install -y software-properties-common build-essential wget libedit-dev libz-dev \
                   python3-yaml python3-pip pkg-config libssl-dev libcurl4-openssl-dev curl \
                   uuid-dev git libffi-dev libmagic-dev \
                   doxygen doxygen-doc doxygen-latex doxygen-gui graphviz \
                   libgflags-dev libncurses-dev \
                   openjdk-8-jdk libyaml-dev ninja-build gcc-10 g++-10 autoconf libtool m4
  
ldconfig
export CC=gcc-10
export CXX=g++-10
'''

In [39]:
print(apt_dependencies())

apt update -y

apt-get install -y software-properties-common build-essential wget libedit-dev libz-dev                    python3-yaml python3-pip pkg-config libssl-dev libcurl4-openssl-dev curl                    uuid-dev git libffi-dev libmagic-dev                    doxygen doxygen-doc doxygen-latex doxygen-gui graphviz                    libgflags-dev libncurses-dev                    openjdk-8-jdk libyaml-dev ninja-build gcc-10 g++-10 autoconf libtool m4
  
ldconfig
export CC=gcc-10
export CXX=g++-10

